# Automatic Setup of Noteboook

The follwing code clones the github repository with the hybrid modeling summer school files. Subsequently it imports all libraries and custom modules needed for this notebook. No need to change anything here.

In [12]:
# import libraries
import numpy as np
import pandas as pd
import scipy
import copy
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import qmc
from scipy.integrate import solve_ivp
from plotly.subplots import make_subplots
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')
pio.templates.default = "simple_white"
pcolors = px.colors.qualitative.T10
pcolors25 = px.colors.qualitative.Alphabet
SEED = np.random.default_rng(42)

# Step 1: Generation of Process Data using a Simulation model

Here we define a simple in-silico process model that we use to generate data to subsequently evaluate which models can approximate the observed behavior. This process model captures only the growth phase of the biomass.

The goal in the following evaluations is to be able to predict the evolution of VCD and Glucose (or just at specific time t) given the initial conditions `VCD_0`, `GLC_0` and a process parameter `FEED_RATE`.

To approximate and predict the VCD and Glucose for unseen parameters we first need to understand the behaviour of the process using some experimental results as training data. Subsequently, this model will allows us to predict the process behaviour for unseen test data.

In [13]:
def ode_fcn(t, y, feed_rate):
  # define parameters
    VCD = y[0]
    Glc = y[1]
    # growth rate
    mu = Glc/(5+Glc)
    # mass balances
    dVCD_dt = mu*VCD
    dGlc_dt = -(0.5*mu+0.05*Glc)*VCD+feed_rate
    dy = [dVCD_dt, dGlc_dt]
    return dy

def run_experiment(VCD_0, Glc_0, feed_rate, t_end, time_step=0.25, noise=0):
    fun = lambda t, y: ode_fcn(t,y,feed_rate)
    y0 = [VCD_0, Glc_0]
    t_span = np.arange(0, 0.5*round(2*t_end)+time_step, time_step)
    sol = solve_ivp(fun, [t_span[0], t_span[-1]], y0, method='LSODA', t_eval=t_span, rtol=1e-6, atol=1e-6)
    t = sol.t.tolist()
    y = sol.y.T
    VCD = y[:, 0] + y[:, 0]*(np.random.rand(len(t))-0.5)*2*noise
    Glc = y[:, 1] + y[:, 1]*(np.random.rand(len(t))-0.5)*2*noise
    return t, VCD, Glc

### Run single experiment to assess process behavior

Here we generate data of an experimental run with the insilico model. The cell process parameters which we are able to manipulate are the following:

*   `VCD_0` Amount of VCD at time 0
*   `Glc_0` Amount of Glc at time 0
*   `feed_rate` Amount of continuous feed each day. Feeding starts at day 1.
*   `feed_end` Day when we stop feeding. This defines also the length of the run.

Subsequently we plot the process evolution over time.


In [14]:
""" Set Process Parameters for run generation """
VCD_0 = .8
GLC_0 = 10
FEED_RATE = 10
FEED_END = 5
""" Measurement granularity """
TIME_STEP = 0.25
""" Measurement noise """
RELATIVE_NOISE = 0.0

In [15]:
# Generate single experiment run
t, VCD, GLC = run_experiment(VCD_0,GLC_0,FEED_RATE,FEED_END,time_step=TIME_STEP,noise=RELATIVE_NOISE)
# Plot single experiment
pcolors = px.colors.qualitative.T10
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=t,y=VCD, mode='markers+lines',name='VCD'))
fig.add_trace(go.Scatter(x=t,y=GLC, mode='markers+lines',name='Glucose'),secondary_y=True)
fig.update_layout(showlegend=True,title="Cell process dynamics",xaxis_title="Time",yaxis_title="Glc",width=1000)
fig.update_yaxes(title_text="VCD", titlefont_color = pcolors[0], tickfont_color=pcolors[0],secondary_y=False)
fig.update_yaxes(title_text="Glc", titlefont_color = pcolors[1], tickfont_color=pcolors[1],secondary_y=True)
fig.show()

### Run experimental design to obtain training data

Here we use the same insilico model as above, but specify ranges for each process parameter. A design of experiments is performed within these parameters ranges using Latin Hypercube Sampling (a non-exhaustive design of experiment approach) for the specified amount of experiments.

This will serve to generate training data for our process models.

In [16]:
""" Number of experiments in training """
NUM_RUNS = 4
""" Define initial conditions for process parameters in DOE """
VCD_0_BOUNDS = [0.5,3.5]
GLC_0_BOUNDS = [10,30]
FEED_RATE_BOUNDS = [10,50]

In [17]:
# Create Design of Experiments
doe_train = np.zeros((NUM_RUNS,3))
sampler = qmc.LatinHypercube(d=3,seed=42)
doe_train_nondim = 2*sampler.random(n=NUM_RUNS)-1
# Transform DOE from [-1,1] range to the specified bounds
doe_train[:,0] = ((doe_train_nondim[:,0]+1)/2)*(VCD_0_BOUNDS[1] - VCD_0_BOUNDS[0]) + VCD_0_BOUNDS[0]
doe_train[:,1] = ((doe_train_nondim[:,1]+1)/2)*(GLC_0_BOUNDS[1] - GLC_0_BOUNDS[0]) + GLC_0_BOUNDS[0]
doe_train[:,2] = ((doe_train_nondim[:,2]+1)/2)*(FEED_RATE_BOUNDS[1] - FEED_RATE_BOUNDS[0]) + FEED_RATE_BOUNDS[0]
LEN_RUNS = int(np.ceil((FEED_END/TIME_STEP)+1))
# Generate experiments
vcd = np.zeros((LEN_RUNS,NUM_RUNS))
glc = np.zeros((LEN_RUNS,NUM_RUNS))
for i in range(NUM_RUNS):
    t, vcd[:,i],glc[:,i] = run_experiment(doe_train[i,0],doe_train[i,1],doe_train[i,2],FEED_END,time_step=TIME_STEP,noise=RELATIVE_NOISE)
# Plot experiments
fig = make_subplots(specs=[[{"secondary_y": True}]])
for i in range(NUM_RUNS):
    fig.add_trace(go.Scatter(x=t,y=vcd[:, i], mode='markers+lines',name=f'VCD | run {i}', line_color=pcolors[0],legendgroup=i))
    fig.add_trace(go.Scatter(x=t,y=glc[:, i], mode='markers+lines',name=f'Glc | run {i}',line_color=pcolors[1],legendgroup=i),secondary_y=True)
fig.update_layout(showlegend=True,title="Cell process dynamics in DOE",xaxis_title="Time",yaxis_title="Glc",width=1000)
fig.update_yaxes(title_text="VCD", titlefont_color = pcolors[0], tickfont_color=pcolors[0],secondary_y=False)
fig.update_yaxes(title_text="Glc", titlefont_color = pcolors[1], tickfont_color=pcolors[1],secondary_y=True)
fig.show()

### Generation of large test set


In similar fashion we create a test set with significantly larger number of experiments to properly verify the model's performance on completely unseen data across the entire process parameter space.

In [18]:
""" Number of experiments in test """
NUM_TEST = 100

In [33]:
# Create design of experiment with more runs for test
doe_test = np.zeros((NUM_TEST,3))
sampler = qmc.LatinHypercube(d=3, seed=42)
doe_test_nondim = 2*sampler.random(n=NUM_TEST)-1
# Transform DOE from [-1,1] range to the specified bounds
doe_test[:,0] = ((doe_test_nondim[:,0]+1)/2)*(VCD_0_BOUNDS[1] - VCD_0_BOUNDS[0]) + VCD_0_BOUNDS[0]
doe_test[:,1] = ((doe_test_nondim[:,1]+1)/2)*(GLC_0_BOUNDS[1] - GLC_0_BOUNDS[0]) + GLC_0_BOUNDS[0]
doe_test[:,2] = ((doe_test_nondim[:,2]+1)/2)*(FEED_RATE_BOUNDS[1] - FEED_RATE_BOUNDS[0]) + FEED_RATE_BOUNDS[0]
LEN_RUNS = int(np.ceil((FEED_END/TIME_STEP)+1))
# Generate experiments
vcd_test = np.zeros((LEN_RUNS,NUM_TEST))
glc_test = np.zeros((LEN_RUNS,NUM_TEST))
for i in range(NUM_TEST):
    t, vcd_test[:,i],glc_test[:,i] = run_experiment(doe_test[i,0],doe_test[i,1],doe_test[i,2],FEED_END,time_step=TIME_STEP,noise=RELATIVE_NOISE)
# Plot experiments
fig = make_subplots(specs=[[{"secondary_y": True}]])
for i in range(NUM_TEST):
    fig.add_trace(go.Scatter(x=t,y=vcd_test[:, i], mode='markers+lines',name=f'VCD | run {i}', line_color=pcolors[0]))
    fig.add_trace(go.Scatter(x=t,y=glc_test[:, i], mode='markers+lines',name=f'Glc | run {i}',line_color=pcolors[1]),secondary_y=True)
fig.update_layout(showlegend=False,title="Cell process dynamics in DOE | Test set",xaxis_title="Time",yaxis_title="Glc",width=1000)
fig.update_yaxes(title_text="VCD", titlefont_color = pcolors[0], tickfont_color=pcolors[0],secondary_y=False)
fig.update_yaxes(title_text="Glc", titlefont_color = pcolors[1], tickfont_color=pcolors[1],secondary_y=True)
fig.show()


# This plot is to be created by the course participants PARTICIPANTS
fig = make_subplots(rows=1,cols=3)
fig.add_trace(go.Scatter(x=doe_train[:,0],y=doe_train[:,1], mode='markers', line_color=pcolors[2], marker_size=14))
fig.add_trace(go.Scatter(x=doe_test[:,0],y=doe_test[:,1], mode='markers', line_color=pcolors[3]))
fig.update_layout(showlegend=False,title="Visualization of the process parameter space",width=1000)
fig.update_xaxes(title_text="VCD0")
fig.update_yaxes(title_text="Glc0")

fig.add_trace(go.Scatter(x=doe_train[:,0],y=doe_train[:,2], mode='markers', line_color=pcolors[2], marker_size=14),row=1,col=2)
fig.add_trace(go.Scatter(x=doe_test[:,0],y=doe_test[:,2], mode='markers', line_color=pcolors[3]),row=1,col=2)
fig.update_xaxes(title_text="VCD0",row=1,col=2)
fig.update_yaxes(title_text="Feed",row=1,col=2)


fig.add_trace(go.Scatter(x=doe_train[:,1],y=doe_train[:,2], mode='markers', line_color=pcolors[2], marker_size=14),row=1,col=3)
fig.add_trace(go.Scatter(x=doe_test[:,1],y=doe_test[:,2], mode='markers', line_color=pcolors[3]),row=1,col=3)
fig.update_xaxes(title_text="Glc0",row=1,col=3)
fig.update_yaxes(title_text="Feed",row=1,col=3)

fig.show()


# Step 2: Data-driven Model

In this section we show how a data-driven model for the insilico data could look like. We create a pure data-driven model that does not try to model the process evolution but directly predicts VCD at time t.


### Train model and use it to predict all the runs

In [20]:
# Define Model
kernel = 1.0 * RBF(length_scale=1e-1, length_scale_bounds=(1e-2, 1e2)) + WhiteKernel(noise_level=1e-2, noise_level_bounds=(1e-10, 1e1))
vcd_mdl = GaussianProcessRegressor(kernel=kernel, random_state=0, alpha=0, n_restarts_optimizer=3)
glc_mdl = GaussianProcessRegressor(kernel=kernel, random_state=0, alpha=0, n_restarts_optimizer=3)

# Train Model
t_idx = [t.index(i) for i in t]
vcd_dd_mdl = []
glc_dd_mdl = []
for i,j in enumerate(t_idx):
    # Each step have copy of the model and is trained independently of others
    vcd_dd_mdl.append(copy.copy(vcd_mdl).fit(doe_train,vcd[j,:]))
    glc_dd_mdl.append(copy.copy(glc_mdl).fit(doe_train,glc[j,:]))

# Prediction in train set
LEN_STEPS = len(t)
vcd_train_pred = np.zeros((LEN_STEPS, NUM_RUNS))
glc_train_pred = np.zeros((LEN_STEPS, NUM_RUNS))
for i in range(NUM_RUNS):
    for j in range(LEN_STEPS):
        vcd_train_pred[j,i] = vcd_dd_mdl[j].predict(doe_train[i,:].reshape(1,-1))
        glc_train_pred[j,i] = glc_dd_mdl[j].predict(doe_train[i,:].reshape(1,-1))

# Prediction in test set
vcd_test_pred = np.zeros((LEN_RUNS,NUM_TEST))
glc_test_pred = np.zeros((LEN_RUNS,NUM_TEST))
for i in range(NUM_TEST):
    for j in range(LEN_STEPS):
        vcd_test_pred[j,i] = vcd_dd_mdl[j].predict(doe_test[i,:].reshape(1,-1))
        glc_test_pred[j,i] = glc_dd_mdl[j].predict(doe_test[i,:].reshape(1,-1))
# Calculate error metrics
vcd_test_rmse = mean_squared_error(vcd_test, vcd_test_pred,squared=False) / np.std(np.array(vcd))
glc_test_rmse = mean_squared_error(glc_test, glc_test_pred,squared=False) / np.std(np.array(vcd))
vcd_test_r2 = r2_score(vcd_test,vcd_test_pred)
glc_test_r2 = r2_score(glc_test,glc_test_pred)

## Evaluate the model performance on all runs

In [21]:
""" Select experiment on which to predict """
PLOT_TRAIN=0
PLOT_TEST=42

# Plot selected experiment run
fig = make_subplots(rows=1,cols=2,specs=[[{"secondary_y": True},{"secondary_y": True}]],subplot_titles=(f"Train set - run {PLOT_TRAIN} - prediction",f"Test set - run {PLOT_TEST} - prediction"))
# Train Set Figures
fig.add_trace(go.Scatter(x=t, y=vcd[:,PLOT_TRAIN], name = 'VCD Observed', line_color=pcolors[0]),row=1,col=1,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=vcd_train_pred[:,PLOT_TRAIN],name='VCD Predicted', line_color=pcolors[0],line_dash='dash'),row=1,col=1,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=glc[:,PLOT_TRAIN], name = 'Glc Observed', line_color=pcolors[1]),row=1,col=1)
fig.add_trace(go.Scatter(x=t, y=glc_train_pred[:,PLOT_TRAIN],name='Glc Predicted', line_color=pcolors[1],line_dash='dash'),row=1,col=1)
# Test Set Figures
fig.add_trace(go.Scatter(x=t, y=vcd_test[:,PLOT_TEST], name = 'VCD Observed', line_color=pcolors[0]),row=1,col=2,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=vcd_test_pred[:,PLOT_TEST],name='VCD Predicted', line_color=pcolors[0],line_dash='dash'),row=1,col=2,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=glc_test[:,PLOT_TEST], name = 'Glc Observed', line_color=pcolors[1]),row=1,col=2)
fig.add_trace(go.Scatter(x=t, y=glc_test_pred[:,PLOT_TEST],name='Glc Predicted', line_color=pcolors[1],line_dash='dash'),row=1,col=2)
fig.update_layout(showlegend=True,title="Predicted Profile: Glc and VCD over time for run id: "+str(PLOT_TRAIN)+" in train set (left) & "+str(PLOT_TEST)+" in test set (right)", height=500)
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text="VCD", titlefont_color = pcolors[0], tickfont_color=pcolors[0],secondary_y=False)
fig.update_yaxes(title_text="Glc", titlefont_color = pcolors[1], tickfont_color=pcolors[1],secondary_y=True)
fig.show()
# Plot observed vs predicted on Test Set
fig = make_subplots(rows=1, cols=2, subplot_titles=[
    f'Test set - all runs - VCD <br> ( rmse = {vcd_test_rmse:.3f}, r2 = {vcd_test_r2:.3f} )',
    f'Test set - all runs - Glc <br> ( rmse = {glc_test_rmse:.3f}, r2 = {glc_test_r2:.3f} )'])
for i in range(NUM_TEST):
    fig.add_trace(go.Scatter(x=vcd_test[:,i],y=vcd_test_pred[:,i],mode='markers',name=f"run id {i}", marker_color=pcolors25[i%25],legendgroup=i),row=1,col=1)
    fig.add_trace(go.Scatter(x=glc_test[:,i],y=glc_test_pred[:,i],mode='markers',name=f"run id {i}", marker_color=pcolors25[i%25],legendgroup=i),row=1,col=2)
fig.add_shape(type="line",x0=vcd_test.min(),y0=vcd_test.min(),x1=vcd_test.max(),y1=vcd_test.max(), layer='above', line_dash='dash',line_width=2,row=1,col=1)
fig.add_shape(type="line",x0=glc_test.min(),y0=glc_test.min(),x1=glc_test.max(),y1=glc_test.max(), layer='above', line_dash='dash',line_width=2,row=1,col=2)
fig.update_layout(title_text = "Observed vs Predicted: ",showlegend=True, height=500)
fig.update_xaxes(title_text='VCD',titlefont_color=pcolors[0], tickfont_color = pcolors[0])
fig.update_yaxes(title_text='VCD',titlefont_color=pcolors[0], tickfont_color = pcolors[0])
fig.update_xaxes(title_text='Glc',titlefont_color=pcolors[1], tickfont_color = pcolors[1],row=1,col=2)
fig.update_yaxes(title_text='Glc',titlefont_color=pcolors[1], tickfont_color = pcolors[1],row=1,col=2)
fig.show()

# Step 3: Hybrid Modeling

We use a simple model for cell expansion, accounting for the growth of viable cell density (VCD) and the consumption of glucose (Glc), to show how hybrid models are trained and can predict new experimental conditions.

Furthermore we define two hybrid models below. The first model defines the ODE equations that describe the dynamics and the second, which is to be evolved by the participants, solves the equations using specified parameters and initial conditions.

## A simple Hybrid Model  

Let us assume the following model (VCD dependent):

$$
\begin{aligned}
&\frac{\mathrm{dVCD}}{\mathrm{dt}}=g(\mathrm{VCD}, \mathrm{Glc}) \\
&\frac{\mathrm{dGlc}}{\mathrm{dt}}=-k(\mathrm{VCD}, \mathrm{Glc})+\text { feed }
\end{aligned}
$$

The functions $g$ and $k$ are to be learned with the machine-learning model and they describe the intrinsic process behavior, like GLC consumption with the total amount of cells. So for GLC it estimates the **global** glucose consumption.

To obtain the full profiles in prediction, we integrate the equations in the following way for each timestep:

$$
VCD_{t + \Delta t} = VCD_{t} + \frac{\mathrm{dVCD}}{\mathrm{dt}}\Delta t
$$


Let us use the inverse (direct) method to get an estimate of the time derivatives for each step individual using the approximation of subsequtive steps:
$$
\frac{\mathrm{dVCD}}{\mathrm{dt}} ≈ \frac{ΔVCD_t }{Δt}  \approx \frac{(VCD_{t + \Delta t} - VCD_{t})}{Δt}
$$

### Estimate the time derivatives

In [22]:
# Estimating derivatives (changes/deltas)
g = np.zeros((LEN_RUNS, NUM_RUNS))
k = np.zeros((LEN_RUNS, NUM_RUNS))
# initial derivatives
g[0,:] = (vcd[1,:]-vcd[0,:])/TIME_STEP
dGlcdt = (glc[1,:]-glc[0,:])/TIME_STEP
k[0,:] = -dGlcdt+doe_train[:,2].T
# timestep derivatives
for j in range(1,LEN_RUNS-1):
    g[j,:] = 0.5*(vcd[j+1,:]-vcd[j-1,:])/TIME_STEP
    dGlcdt = 0.5*(glc[j+1,:]-glc[j-1,:])/TIME_STEP
    k[j,:] = -dGlcdt+doe_train[:,2].T
# final derivatives
g[-1,:] = (vcd[-1,:]-vcd[-2,:])/TIME_STEP
dGlcdt = (glc[-1,:]-glc[-2,:])/TIME_STEP
k[-1,:] = -dGlcdt+doe_train[:,2].T

In [23]:
# Plot scatterplots for VCD and GLC, color by derivative
fig = make_subplots(rows=2, cols=2, subplot_titles=("VCD over time - VCD growth rate","GLC over time - GLC consumption rate","VCD vs GLC - VCD growth rate","VCD vc GLC - GLC consumption rate" ))
fig.add_trace(go.Scatter(x=np.repeat(t,NUM_RUNS),y=vcd.flatten(),mode='markers',marker=dict(color=g.flatten(),colorscale=px.colors.sequential.Viridis,showscale=True,colorbar=dict(len=1.0, x=0.45)),),row=1,col=1)
fig.add_trace(go.Scatter(x=np.repeat(t,NUM_RUNS),y=glc.flatten(),mode='markers',marker=dict(color=k.flatten(),colorscale=px.colors.sequential.Viridis,showscale=True,colorbar=dict(len=1.0, x=1.0)),),row=1,col=2)
fig.add_trace(go.Scatter(x=vcd.flatten(),y=glc.flatten(),mode='markers',marker=dict(color=g.flatten(),colorscale=px.colors.sequential.Viridis),),row=2,col=1)
fig.add_trace(go.Scatter(x=vcd.flatten(),y=glc.flatten(),mode='markers',marker=dict(color=k.flatten(),colorscale=px.colors.sequential.Viridis),),row=2,col=2)
fig.update_layout(title_text = "State plots of process colored by growth/consumption rates",showlegend=False,height=1000)
fig.update_xaxes(title_text="Time", row=1, col=1), fig.update_yaxes(title_text="VCD", titlefont_color=pcolors[0], tickfont_color = pcolors[0], row=1, col=1)
fig.update_xaxes(title_text="Time", row=1, col=2), fig.update_yaxes(title_text="GLC", titlefont_color=pcolors[1], tickfont_color = pcolors[1], row=1, col=2)
fig.update_xaxes(title_text="VCD", titlefont_color=pcolors[0], tickfont_color = pcolors[0], row=2, col=1), fig.update_yaxes(title_text="GLC", titlefont_color=pcolors[1], tickfont_color = pcolors[1], row=2, col=1)
fig.update_xaxes(title_text="VCD", titlefont_color=pcolors[0], tickfont_color = pcolors[0], row=2, col=2), fig.update_yaxes(title_text="GLC", titlefont_color=pcolors[1], tickfont_color = pcolors[1], row=2, col=2)
fig.show()

### Train the hybrid model and use it to predict all runs

Let us fit the same data-driven model structure as above, but change its outputs to the estimated derivatives:

In [24]:
# Define Model
kernel = 1.0 * RBF(length_scale=1e-1, length_scale_bounds=(1e-2, 1e2)) + WhiteKernel(noise_level=1e-2, noise_level_bounds=(1e-10, 1e1))
g_1h_mdl = GaussianProcessRegressor(kernel=kernel, random_state=0, alpha=0, n_restarts_optimizer=3)
k_1h_mdl = GaussianProcessRegressor(kernel=kernel, random_state=0, alpha=0, n_restarts_optimizer=3)

def ode_fcn_1st_hybrid(t, y, feed, g_mdl, k_mdl):
    """
    --- Inputs ---
    t: Current timestep of the process
    y: Current states of VCD and Glucose
    feed: Feed rate for the experiment
    g_mdl, k_mdl: Models for derivatives of growth and consumption rate
                  (sklearn expects 2D array, even when predicting on a single observation, use .reshape(1, -1))
    --- Outputs ---
    dVCD_dt, dGlc_df : Derivatives of VCD and Glucose for the next state from current one
    """
    # mass balances
    dVCD_dt = g_mdl.predict(y.reshape(1, -1))
    dGlc_dt = -k_mdl.predict(y.reshape(1, -1)) + feed
    return [dVCD_dt, dGlc_dt]

def run_1st_hybrid(VCD_0, Glc_0, feed, t_end, g_mdl, k_mdl,time_step =0.25):
    """
    --- Inputs ---
    VCD_0, Glc_0: Initial conditions
    feed: Feed rate for the experiment
    t_end: End time of feed/experiment
    g_mdl, k_mdl: Models for derivatives of growth and consumption rate
    --- Outputs ---
    t: array of timesteps
    VCD, Glc: array of VCD/Glc values as evolution over the experiment run
    """
    # Get initial states / values
    y0 = np.array([VCD_0, Glc_0])
    # Get all the time-steps on which we want to predict
    t_span = np.arange(0, 0.5 * round(2 * t_end) + time_step, time_step)
    # Define a function that predicts derivative
    fun = lambda t, y: ode_fcn_1st_hybrid(t, y, feed, g_mdl, k_mdl)
    # Use ODE solver to obtain the profile
    sol = solve_ivp(fun, [t_span[0], t_span[-1]], y0, method='LSODA', t_eval=t_span, rtol=1e-6, atol=1e-6)
    # Collect results
    t = sol.t.tolist()
    y = sol.y.T
    VCD = y[:, 0]
    Glc = y[:, 1]
    return t, VCD, Glc

# Define inputs as 2 array of VCD and Glc states
X = np.stack((vcd.flatten(), glc.flatten()), axis=1)

# Train Model
g_1h_mdl.fit(X, g.flatten())
k_1h_mdl.fit(X, k.flatten())

# Predictions in train set
vcd_train_pred = np.zeros((LEN_RUNS, NUM_RUNS))
glc_train_pred = np.zeros((LEN_RUNS, NUM_RUNS))
for i in range(NUM_RUNS):
    t,vcd_train_pred[:,i], glc_train_pred[:,i] = run_1st_hybrid(doe_train[i,0], doe_train[i,1], doe_train[i,2], FEED_END, g_1h_mdl, k_1h_mdl, time_step=TIME_STEP)
# Predictions in test set
vcd_test_pred = np.zeros((LEN_RUNS,NUM_TEST))
glc_test_pred = np.zeros((LEN_RUNS,NUM_TEST))
for i in range(NUM_TEST):
    t, vcd_test_pred[:,i], glc_test_pred[:,i] = run_1st_hybrid(doe_test[i,0], doe_test[i,1], doe_test[i,2], FEED_END, g_1h_mdl, k_1h_mdl, time_step=TIME_STEP)
# Calculate error metrics
vcd_test_rmse = mean_squared_error(vcd_test, vcd_test_pred,squared=False) / np.std(np.array(vcd))
glc_test_rmse = mean_squared_error(glc_test, glc_test_pred,squared=False) / np.std(np.array(vcd))
vcd_test_r2 = r2_score(vcd_test,vcd_test_pred)
glc_test_r2 = r2_score(glc_test,glc_test_pred)

### Model Evaluation on Train and Test Set

In [25]:
""" Select experiment on which to predict """
PLOT_TRAIN=0
PLOT_TEST=42
# Plot selected experiment run
fig = make_subplots(rows=1,cols=2,specs=[[{"secondary_y": True},{"secondary_y": True}]],subplot_titles=(f"Train set - run {PLOT_TRAIN} - prediction",f"Test set - run {PLOT_TEST} - prediction"))
# Train Set Figures
fig.add_trace(go.Scatter(x=t, y=vcd[:,PLOT_TRAIN], name = 'VCD Observed', line_color=pcolors[0]),row=1,col=1,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=vcd_train_pred[:,PLOT_TRAIN],name='VCD Predicted', line_color=pcolors[0],line_dash='dash'),row=1,col=1,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=glc[:,PLOT_TRAIN], name = 'Glc Observed', line_color=pcolors[1]),row=1,col=1)
fig.add_trace(go.Scatter(x=t, y=glc_train_pred[:,PLOT_TRAIN],name='Glc Predicted', line_color=pcolors[1],line_dash='dash'),row=1,col=1)
# Test Set Figures
fig.add_trace(go.Scatter(x=t, y=vcd_test[:,PLOT_TEST], name = 'VCD Observed', line_color=pcolors[0]),row=1,col=2,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=vcd_test_pred[:,PLOT_TEST],name='VCD Predicted', line_color=pcolors[0],line_dash='dash'),row=1,col=2,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=glc_test[:,PLOT_TEST], name = 'Glc Observed', line_color=pcolors[1]),row=1,col=2)
fig.add_trace(go.Scatter(x=t, y=glc_test_pred[:,PLOT_TEST],name='Glc Predicted', line_color=pcolors[1],line_dash='dash'),row=1,col=2)
fig.update_layout(showlegend=True,title="Predicted Profile: Glc and VCD over time for run id: "+str(PLOT_TRAIN)+" in train set (left) & "+str(PLOT_TEST)+" in test set (right)", height=500)
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text="VCD", titlefont_color = pcolors[0], tickfont_color=pcolors[0],secondary_y=False)
fig.update_yaxes(title_text="Glc", titlefont_color = pcolors[1], tickfont_color=pcolors[1],secondary_y=True)
fig.show()
# Plot observed vs predicted on Test Set
fig = make_subplots(rows=1, cols=2, subplot_titles=[
    f'Test set - all runs - VCD <br> ( rmse = {vcd_test_rmse:.3f}, r2 = {vcd_test_r2:.3f} )',
    f'Test set - all runs - Glc <br> ( rmse = {glc_test_rmse:.3f}, r2 = {glc_test_r2:.3f} )'])
for i in range(NUM_TEST):
    fig.add_trace(go.Scatter(x=vcd_test[:,i],y=vcd_test_pred[:,i],mode='markers',name=f"run id {i}", marker_color=pcolors25[i%25],legendgroup=i),row=1,col=1)
    fig.add_trace(go.Scatter(x=glc_test[:,i],y=glc_test_pred[:,i],mode='markers',name=f"run id {i}", marker_color=pcolors25[i%25],legendgroup=i),row=1,col=2)
fig.add_shape(type="line",x0=vcd_test.min(),y0=vcd_test.min(),x1=vcd_test.max(),y1=vcd_test.max(), layer='above', line_dash='dash',line_width=2,row=1,col=1)
fig.add_shape(type="line",x0=glc_test.min(),y0=glc_test.min(),x1=glc_test.max(),y1=glc_test.max(), layer='above', line_dash='dash',line_width=2,row=1,col=2)
fig.update_layout(title_text = "Observed vs Predicted: ",showlegend=True, height=500)
fig.update_xaxes(title_text='VCD',titlefont_color=pcolors[0], tickfont_color = pcolors[0])
fig.update_yaxes(title_text='VCD',titlefont_color=pcolors[0], tickfont_color = pcolors[0])
fig.update_xaxes(title_text='Glc',titlefont_color=pcolors[1], tickfont_color = pcolors[1],row=1,col=2)
fig.update_yaxes(title_text='Glc',titlefont_color=pcolors[1], tickfont_color = pcolors[1],row=1,col=2)
fig.show()

We can see that the model performs generally very good, but a number of runs in the test set are not well predicted. Why is that?

## Improving on the simple Hybrid Model

Let us assumed the following model (VCD independent):

$$
\begin{aligned}
&\frac{\mathrm{dVCD}}{\mathrm{dt}}=\mu(\mathrm{VCD}, \mathrm{Glc}) \cdot \mathrm{VCD} \\
&\frac{\mathrm{dGlc}}{\mathrm{dt}}=-v(\mathrm{VCD}, \mathrm{Glc}) \cdot \mathrm{VCD}+\text { feed }
\end{aligned}
$$

Here the functions $\mu$ and $v$ are the specific growth and glucose consumption rates. These rates need to be multiplied to by the overall amount of cells in the process to obtain the total growth and consumption rates.


To obtain the full profiles in prediction, we integrate as before in the following way for each timestep:

$$
VCD_{t + \Delta t} = VCD_{t} + \frac{\mathrm{dVCD}}{\mathrm{dt}}\Delta t
$$


As before, let us use the inverse (direct) method to estimate the time derivatives for each step individual using the approximation of subsequtive steps:
$$
\frac{\mathrm{dVCD}}{\mathrm{dt}} ≈ \frac{ΔVCD_t }{Δt}  \approx \frac{(VCD_{t + \Delta t} - VCD_{t})}{Δt}
$$


### Estimate the time derivatives taking the knowledge additions into account

In [26]:
# Estimating derivatives (changes/deltas)
mu = np.zeros((LEN_RUNS, NUM_RUNS))
v = np.zeros((LEN_RUNS, NUM_RUNS))
# initial derivatives
dVcddt = (vcd[1,:]-vcd[0,:])/TIME_STEP
mu[0,:] = dVcddt/vcd[0,:]
dGlcdt = (glc[1,:]-glc[0,:])/TIME_STEP
v[0,:] = (-dGlcdt+doe_train[:,2].T)/vcd[1,:]
# timestep derivatives
for j in range(1,LEN_RUNS-1):
    dVcddt = 0.5*(vcd[j+1,:]-vcd[j-1,:])/TIME_STEP
    mu[j,:] = dVcddt/vcd[j,:]
    dGlcdt = 0.5*(glc[j+1,:]-glc[j-1,:])/TIME_STEP
    v[j,:] = (-dGlcdt+doe_train[:,2].T)/vcd[j,:]
# final derivatives
dVcddt = (vcd[-1,:]-vcd[-2,:])/TIME_STEP
mu[-1,:] = dVcddt/vcd[-1,:]
dGlcdt = (glc[-1,:]-glc[-2,:])/TIME_STEP
v[-1,:] = (-dGlcdt+doe_train[:,2].T)/vcd[-1,:]

In [27]:
# Plot scatterplots for VCD and GLC, color by derivative
fig = make_subplots(rows=2, cols=2, subplot_titles=("VCD over time - VCD spec growth rate","GLC over time - GLC spec consumption rate","VCD vs GLC - VCD spec growth rate","VCD vc GLC - GLC spec consumption rate" ))
fig.add_trace(go.Scatter(x=np.repeat(t,NUM_RUNS),y=vcd.flatten(),mode='markers',marker=dict(color=mu.flatten(),colorscale=px.colors.sequential.Viridis,showscale=True,colorbar=dict(len=1.0, x=0.45)),),row=1,col=1)
fig.add_trace(go.Scatter(x=np.repeat(t,NUM_RUNS),y=glc.flatten(),mode='markers',marker=dict(color=v.flatten(),colorscale=px.colors.sequential.Viridis,showscale=True,colorbar=dict(len=1.0, x=1.0)),),row=1,col=2)
fig.add_trace(go.Scatter(x=vcd.flatten(),y=glc.flatten(),mode='markers',marker=dict(color=mu.flatten(),colorscale=px.colors.sequential.Viridis),),row=2,col=1)
fig.add_trace(go.Scatter(x=vcd.flatten(),y=glc.flatten(),mode='markers',marker=dict(color=v.flatten(),colorscale=px.colors.sequential.Viridis),),row=2,col=2)
fig.update_layout(title_text = "State plots of process colored by growth/consumption rates",showlegend=False,height=1000)
fig.update_xaxes(title_text="Time", row=1, col=1), fig.update_yaxes(title_text="VCD", titlefont_color=pcolors[0], tickfont_color = pcolors[0], row=1, col=1)
fig.update_xaxes(title_text="Time", row=1, col=2), fig.update_yaxes(title_text="GLC", titlefont_color=pcolors[1], tickfont_color = pcolors[1], row=1, col=2)
fig.update_xaxes(title_text="VCD", titlefont_color=pcolors[0], tickfont_color = pcolors[0], row=2, col=1), fig.update_yaxes(title_text="GLC", titlefont_color=pcolors[1], tickfont_color = pcolors[1], row=2, col=1)
fig.update_xaxes(title_text="VCD", titlefont_color=pcolors[0], tickfont_color = pcolors[0], row=2, col=2), fig.update_yaxes(title_text="GLC", titlefont_color=pcolors[1], tickfont_color = pcolors[1], row=2, col=2)
fig.show()

### Train the improved hybrid model and use it to predict all runs
Now let us define the improved hybrid model:

In [28]:
# Define Model
kernel = 1.0 * RBF(length_scale=1e-1, length_scale_bounds=(1e-2, 1e2)) + WhiteKernel(noise_level=1e-2, noise_level_bounds=(1e-10, 1e1))
mu_2h_mdl = GaussianProcessRegressor(kernel=kernel, random_state=0, alpha=0, n_restarts_optimizer=3)
v_2h_mdl = GaussianProcessRegressor(kernel=kernel, random_state=0, alpha=0, n_restarts_optimizer=3)

def ode_fcn_2nd_hybrid(t, y, feed, mu_mdl, v_mdl):
    """
    --- Inputs ---
    t: Current timestep of the process
    y: Current states of VCD and Glucose
    feed: Feed rate for the experiment
    g_mdl, k_mdl: Models for derivatives of growth and consumption rate
                  (sklearn model expects 2D array, even when predicting on a single observation, use .reshape(1, -1))
    --- Outputs ---
    dVCD_dt, dGlc_df : Derivatives of VCD and Glucose for the next state from current one
    """
    # mass balances
    dVCD_dt = mu_mdl.predict(y.reshape(1, -1))*y[0]
    dGlc_dt = -v_mdl.predict(y.reshape(1, -1))*y[0]+feed
    return [dVCD_dt, dGlc_dt]


def run_2nd_hybrid(VCD_0,Glc_0,feed,t_end,mu_mdl,v_mdl,time_step=0.25):
    # Get initial states / values
    y0 = np.array([VCD_0, Glc_0])
    # Get all the time-steps on which we want to predict
    t_span = np.arange(0, 0.5 * round(2 * t_end) + time_step, time_step)
    # Define a function that predicts derivative
    fun = lambda t, y: ode_fcn_2nd_hybrid(t, y, feed, mu_mdl, v_mdl)
    # Use ODE solver to obtain the profile
    sol = solve_ivp(fun, [t_span[0], t_span[-1]], y0, method='LSODA', t_eval=t_span, rtol=1e-6, atol=1e-6)
    # Collect results
    t = sol.t.tolist()
    y = sol.y.T
    VCD = y[:, 0]
    Glc = y[:, 1]
    return t, VCD, Glc

# Define inputs as 2 array of VCD and Glc states
X = np.stack((vcd.flatten(), glc.flatten()), axis=1)

# Train Model
mu_2h_mdl.fit(X, mu.flatten())
v_2h_mdl.fit(X, v.flatten())

# Predictions in train set
vcd_train_pred = np.zeros((LEN_RUNS, NUM_RUNS))
glc_train_pred = np.zeros((LEN_RUNS, NUM_RUNS))
for i in range(NUM_RUNS):
    t,vcd_train_pred[:,i], glc_train_pred[:,i] = run_2nd_hybrid(doe_train[i,0], doe_train[i,1], doe_train[i,2], FEED_END, mu_2h_mdl, v_2h_mdl, time_step=TIME_STEP)
# Predictions in test set
vcd_test_pred = np.zeros((LEN_RUNS,NUM_TEST))
glc_test_pred = np.zeros((LEN_RUNS,NUM_TEST))
for i in range(NUM_TEST):
    t, vcd_test_pred[:,i], glc_test_pred[:,i] = run_2nd_hybrid(doe_test[i,0], doe_test[i,1], doe_test[i,2], FEED_END, mu_2h_mdl, v_2h_mdl, time_step=TIME_STEP)

vcd_test_rmse = mean_squared_error(vcd_test, vcd_test_pred,squared=False) / np.std(np.array(vcd))
glc_test_rmse = mean_squared_error(glc_test, glc_test_pred,squared=False) / np.std(np.array(vcd))
vcd_test_r2 = r2_score(vcd_test,vcd_test_pred)
glc_test_r2 = r2_score(glc_test,glc_test_pred)


### Model Evaluation on Train and Test Set



Plot the fit of the improved hybrid model over time

In [29]:
""" Select experiment on which to predict """
PLOT_TRAIN=0
PLOT_TEST=42
# Plot selected experiment run
fig = make_subplots(rows=1,cols=2,specs=[[{"secondary_y": True},{"secondary_y": True}]],subplot_titles=(f"Train set - run {PLOT_TRAIN} - prediction",f"Test set - run {PLOT_TEST} - prediction"))
# Train Set Figures
fig.add_trace(go.Scatter(x=t, y=vcd[:,PLOT_TRAIN], name = 'VCD Observed', line_color=pcolors[0]),row=1,col=1,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=vcd_train_pred[:,PLOT_TRAIN],name='VCD Predicted', line_color=pcolors[0],line_dash='dash'),row=1,col=1,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=glc[:,PLOT_TRAIN], name = 'Glc Observed', line_color=pcolors[1]),row=1,col=1)
fig.add_trace(go.Scatter(x=t, y=glc_train_pred[:,PLOT_TRAIN],name='Glc Predicted', line_color=pcolors[1],line_dash='dash'),row=1,col=1)
# Test Set Figures
fig.add_trace(go.Scatter(x=t, y=vcd_test[:,PLOT_TEST], name = 'VCD Observed', line_color=pcolors[0]),row=1,col=2,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=vcd_test_pred[:,PLOT_TEST],name='VCD Predicted', line_color=pcolors[0],line_dash='dash'),row=1,col=2,secondary_y=True)
fig.add_trace(go.Scatter(x=t, y=glc_test[:,PLOT_TEST], name = 'Glc Observed', line_color=pcolors[1]),row=1,col=2)
fig.add_trace(go.Scatter(x=t, y=glc_test_pred[:,PLOT_TEST],name='Glc Predicted', line_color=pcolors[1],line_dash='dash'),row=1,col=2)
fig.update_layout(showlegend=True,title="Predicted Profile: Glc and VCD over time for run id: "+str(PLOT_TRAIN)+" in train set (left) & "+str(PLOT_TEST)+" in test set (right)", height=500)
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text="VCD", titlefont_color = pcolors[0], tickfont_color=pcolors[0],secondary_y=False)
fig.update_yaxes(title_text="Glc", titlefont_color = pcolors[1], tickfont_color=pcolors[1],secondary_y=True)
fig.show()
# Plot observed vs predicted
fig = make_subplots(rows=1, cols=2, subplot_titles=[
    f'Test set - all runs - VCD <br> ( rmse = {vcd_test_rmse:.3f}, r2 = {vcd_test_r2:.3f} )',
    f'Test set - all runs - Glc <br> ( rmse = {glc_test_rmse:.3f}, r2 = {glc_test_r2:.3f} )'])
for i in range(NUM_TEST):
    fig.add_trace(go.Scatter(x=vcd_test[:,i],y=vcd_test_pred[:,i],mode='markers',name=f"run id {i}", marker_color=pcolors25[i%25],legendgroup=i),row=1,col=1)
    fig.add_trace(go.Scatter(x=glc_test[:,i],y=glc_test_pred[:,i],mode='markers',name=f"run id {i}", marker_color=pcolors25[i%25],legendgroup=i),row=1,col=2)
fig.add_shape(type="line",x0=vcd_test.min(),y0=vcd_test.min(),x1=vcd_test.max(),y1=vcd_test.max(), layer='above', line_dash='dash',line_width=2,row=1,col=1)
fig.add_shape(type="line",x0=glc_test.min(),y0=glc_test.min(),x1=glc_test.max(),y1=glc_test.max(), layer='above', line_dash='dash',line_width=2,row=1,col=2)
fig.update_layout(title_text = "Observed vs Predicted on Test Set",showlegend=True, height=500)
fig.update_xaxes(title_text='VCD',titlefont_color=pcolors[0], tickfont_color = pcolors[0])
fig.update_yaxes(title_text='VCD',titlefont_color=pcolors[0], tickfont_color = pcolors[0])
fig.update_xaxes(title_text='Glc',titlefont_color=pcolors[1], tickfont_color = pcolors[1],row=1,col=2)
fig.update_yaxes(title_text='Glc',titlefont_color=pcolors[1], tickfont_color = pcolors[1],row=1,col=2)
fig.show()

# Step 4: Discussion on the Advantages and disadvantages of hybrid model

Pros:
*   Works well with small dataset
*   Describes the full dynamics
*   Extrapolation possible
*   Incorporate domain knowledge
*   ...


Cons:
*   Prone to failing due data mistakes
*   Units must be consistent
*   Prone to process dynamic misspecification
*   ...
